In [1]:
import os
import json
import networkx as nx
from networkx.readwrite import json_graph
from config import DATA_DIR

def load(outdir: str, graph_type: str = 'chunk',verbose=1):
    """
    Load a graph from disk and replace the internal knowledge graph.
    Args:
        path: Path to the graph JSON file.
    """
    if graph_type == 'chunk':
        filename = "chunk_knowledge_graph.json"
        file_path = os.path.join(f"{outdir}/knowledge_graph", filename)
    elif graph_type == 'page':
        filename = "page_knowledge_graph.json"
        file_path = os.path.join(f"{outdir}/knowledge_graph", filename)
    else:
        raise ValueError("graph_type must be 'chunk' or 'page'")

    with open(file_path, 'r') as f:
        data = json.load(f)

    if graph_type == 'chunk':
        graph = json_graph.node_link_graph(data)
    elif graph_type == 'page':
        graph = json_graph.node_link_graph(data)
    else:
        raise ValueError("graph_type must be 'chunk' or 'page'")  
    
    if verbose:
        print(f"Graph loaded from {file_path}")

    return graph

In [2]:
G = load(DATA_DIR,'chunk')
print("Chunk Knowledge Graph Nodes: ",len(G.nodes))
print("Chunk Knowledge Graph Edges: ",len(G.edges))

/home/achraf/miniconda3/envs/rapids-25.04/lib/python3.12/site-packages/networkx/readwrite/json_graph/node_link.py:287: FutureWarning: 
The default value will be changed to `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_graph(data, edges="links") to preserve current behavior, or
  nx.node_link_graph(data, edges="edges") for forward compatibility.
  warnings.warn(


Graph loaded from data//knowledge_graph/chunk_knowledge_graph.json
Chunk Knowledge Graph Nodes:  33311
Chunk Knowledge Graph Edges:  1230098


In [3]:
import cudf
import cugraph

# Convert NetworkX DiGraph to cuGraph format
def nx_to_cugraph(G):
    edges = nx.to_pandas_edgelist(G)
    gdf_edges = cudf.DataFrame.from_pandas(edges)
    # Rename for cuGraph convention
    gdf_edges = gdf_edges.rename(columns={'source': 'src', 'target': 'dst'})
    return gdf_edges

In [4]:
edges_df = nx_to_cugraph(G)
edges_df

,src,dst,weight,label
0,%C3%89poni_2_0,Vinsmoke_Family_5_0,0.671011,Vinsmoke Family
1,%C3%89poni_2_0,Vinsmoke_Family_6_0,0.671011,Vinsmoke Family
2,%C3%89poni_2_0,Vinsmoke_Family_16_0,0.653096,Vinsmoke Family
3,%C3%89poni_2_0,Sanji_2_0,0.525917,Sanji
4,%C3%89poni_2_0,Sanji_4_0,0.505482,Sanji
...,...,...,...,...
1230093,Z_34_0,Pirate_4_0,0.599726,Roger's era
1230094,Z_34_0,Pirate_18_0,0.598633,Roger's era
1230095,Z_34_0,Buggy_31_0,0.674584,Buggy
1230096,Z_34_0,Buggy_30_0,0.671672,Buggy


In [5]:
G_cu = cugraph.Graph()
G_cu.from_cudf_edgelist(edges_df, source='src', destination='dst', edge_attr='weight', renumber=True)

In [6]:
pagerank_df = cugraph.pagerank(G_cu)

/home/achraf/miniconda3/envs/rapids-25.04/lib/python3.12/site-packages/cugraph/link_analysis/pagerank.py:232: UserWarning: Pagerank expects the 'store_transposed' flag to be set to 'True' for optimal performance during the graph creation
  warnings.warn(warning_msg, UserWarning)


In [7]:
pagerank_df.sort_values("pagerank", ascending=False)

,pagerank,vertex
3600,0.002452,Monkey_D._Luffy_2_1
3601,0.002248,Monkey_D._Luffy_1_1
3615,0.002035,Monkey_D._Luffy_4_0
3603,0.001558,Straw_Hat_Pirates_5_0
3602,0.001144,Sanji_4_0
...,...,...
29335,0.000005,Misery_2_0
30552,0.000005,Hyouzou_16_0
28762,0.000005,Enishida_7_0
30486,0.000005,Trafalgar_D._Water_Law/Misc._8_0


In [8]:
betweenness_df = cugraph.betweenness_centrality(G_cu, k=1024, normalized=True)

In [9]:
betweenness_df.sort_values("betweenness_centrality", ascending=False)

,betweenness_centrality,vertex
3600,0.061097,Monkey_D._Luffy_2_1
3601,0.035997,Monkey_D._Luffy_1_1
3615,0.030105,Monkey_D._Luffy_4_0
3603,0.019069,Straw_Hat_Pirates_5_0
3613,0.018226,Donquixote_Doflamingo_1_0
...,...,...
31017,0.000000,Urban_1_0
31018,0.000000,Charlotte_Cinnamon_11_0
31021,0.000000,Ahho_Zurako_5_0
31024,0.000000,Nerine_2_0


In [10]:
edge_betweenness_df = cugraph.edge_betweenness_centrality(G_cu, k=256)

In [11]:
edge_betweenness_df.sort_values("betweenness_centrality", ascending=False)

,src,dst,betweenness_centrality
734164,Coburn_1_0,Shandia_Chief_4_0,0.004002
123420,Coburn_1_0,Shandia_Chief_1_0,0.003938
267421,Fish-Men_3_0,Tansui_14_0,0.003938
778025,Ginko_4_0,Kin%27emon/History_4_1,0.003938
974946,Charlotte_Family_1_0,Charlotte_Mond%C3%A9e_11_0,0.003938
...,...,...,...
1162712,Hiroshi_5_0,Wakasa_4_0,0.000000
1165771,Scales_6_0,Underbite_7_0,0.000000
1169950,Hawkman_7_0,Koshi_Falcon_6_0,0.000000
1182464,Dirt_Boss_2_0,Forest_Boss_5_0,0.000000


In [12]:
parts, _ = cugraph.louvain(G_cu)

In [13]:
parts

,partition,vertex
0,0,Nami_4_0
1,1,New_World_8_0
2,0,Kaidou_42_0
3,1,Admiral_4_0
4,1,New_World_7_0
...,...,...
31028,6,Raijin_3_0
31029,6,Blackback_1_0
31030,9,Tony_Tony_Chopper/History/During_and_After_the...
31031,2,Cabaji_7_0


In [14]:
def filter_nodes_by_hybrid_score(pagerank_df, betweenness_df, p=0.8):
    """
    Combines PageRank and Betweenness into a hybrid score, keeps top-K nodes.
    """
    merged = pagerank_df.merge(betweenness_df, on='vertex', suffixes=('_pr', '_bc'))
    merged['hybrid'] = merged['pagerank'] * merged['betweenness_centrality']
    top_k = int(len(merged) * p)
    top_nodes = merged.nlargest(top_k, 'hybrid')['vertex']
    return top_nodes

In [15]:
def filter_edges_by_edge_betweenness(edges_df, edge_betweenness_df, min_score=0.00001):
    """
    Removes edges below edge betweenness threshold.
    """
    filtered = edges_df.merge(edge_betweenness_df, on=['src', 'dst'])
    filtered = filtered[filtered['betweenness_centrality'] >= min_score]
    return filtered

In [16]:
top_nodes = filter_nodes_by_hybrid_score(pagerank_df, betweenness_df, p=0.8)

In [17]:
top_nodes

960          Monkey_D._Luffy_2_1
961          Monkey_D._Luffy_1_1
975          Monkey_D._Luffy_4_0
963        Straw_Hat_Pirates_5_0
962                    Sanji_4_0
                  ...           
26143    Charlotte_Dacquoise_5_0
26596       Charlotte_Basans_5_0
23300       Boa_Sandersonia_11_0
24930           Foxy_Fighter_3_0
27295           Foxy_Fighter_4_0
Name: vertex, Length: 24826, dtype: object

In [18]:
edges_df = edges_df[edges_df['src'].isin(top_nodes) & edges_df['dst'].isin(top_nodes)]

In [19]:
edges_df

,src,dst,weight,label
6,%C3%89poni_4_0,Germa_Kingdom_9_0,0.724194,Germa Kingdom
7,%C3%89poni_4_0,Germa_Kingdom_8_0,0.693308,Germa Kingdom
8,%C3%89poni_4_0,Germa_Kingdom_13_0,0.678200,Germa Kingdom
9,%C3%89poni_4_0,Queen_(Title)_4_0,0.658106,queen
10,%C3%89poni_4_0,Queen_(Title)_6_0,0.636521,queen
...,...,...,...,...
1230093,Z_34_0,Pirate_4_0,0.599726,Roger's era
1230094,Z_34_0,Pirate_18_0,0.598633,Roger's era
1230095,Z_34_0,Buggy_31_0,0.674584,Buggy
1230096,Z_34_0,Buggy_30_0,0.671672,Buggy


In [20]:
import cudf

def remove_dead_ends_and_orphans(edges_cudf, recurse=False, verbose=True):
    """
    Removes edges where the source or target node is an orphan or dead-end.
    A node is an orphan/dead-end if it appears only as a source or only as a target.

    Parameters:
    - edges_cudf: cudf.DataFrame with ['src', 'dst']
    - recurse: if True, repeats the pruning until convergence
    - verbose: if True, prints number of edges removed

    Returns:
    - cudf.DataFrame with cleaned edges
    """
    edges = edges_cudf.copy(deep=True)
    total_removed = 0

    while True:
        src_nodes = edges['src'].dropna().drop_duplicates()
        dst_nodes = edges['dst'].dropna().drop_duplicates()

        # Compute node differences (orphans & dead-ends)
        orphan_nodes = src_nodes[~src_nodes.isin(dst_nodes)]
        dead_end_nodes = dst_nodes[~dst_nodes.isin(src_nodes)]

        nodes_to_remove = cudf.concat([orphan_nodes, dead_end_nodes], ignore_index=True)

        if nodes_to_remove.empty:
            break

        before = len(edges)
        mask = ~edges['src'].isin(nodes_to_remove) & ~edges['dst'].isin(nodes_to_remove)
        edges = edges[mask]
        removed = before - len(edges)
        total_removed += removed

        if verbose:
            print(f"Removed {removed} edges...")

        if not recurse:
            break

    if verbose:
        print(f"Total edges removed: {total_removed}")

    return edges


In [21]:
edges_df = remove_dead_ends_and_orphans(edges_df, recurse=True)

Removed 436765 edges...
Removed 31519 edges...
Removed 4478 edges...
Removed 231 edges...
Removed 26 edges...
Removed 60 edges...
Total edges removed: 473079


In [22]:
edges_df = filter_edges_by_edge_betweenness(edges_df, edge_betweenness_df, min_score=0.00001)

In [23]:
edges_df

,src,dst,weight,label,betweenness_centrality
0,Sword_6_0,Wano_Country_24_0,0.314822,Wano Country,0.000032
12,Elizabello_II_2_0,Usopp_1_0,0.529849,Usopp,0.000027
35,Fukaboshi_18_0,Monkey_D._Luffy_2_1,0.408045,Luffy,0.000010
52,Blackbeard_Pirates_6_0,New_World_1_0,0.685822,New World,0.000012
67,Hakumai_8_0,Roronoa_Zoro_4_0,0.562272,Roronoa Zoro,0.000015
...,...,...,...,...,...
375010,Donquixote_Family_9_0,Monkey_D._Luffy_1_1,0.653494,Monkey D. Luffy,0.000011
375077,Four_Emperors_19_1,Queen_12_0,0.628351,Queen,0.000027
375128,Morgans_13_0,Underworld_8_0,0.629842,Underworld,0.000013
375218,Enel_6_0,Usopp_2_0,0.537219,Usopp,0.000011


In [24]:
nodes_df = cudf.concat([
        edges_df['src'], edges_df['dst']
    ]).drop_duplicates().reset_index(drop=True).to_frame(name='node')

nodes_df = nodes_df.merge(parts, left_on='node', right_on='vertex', how='left')
nodes_df = nodes_df.drop(columns=['vertex'])

In [25]:
nodes_df

,node,partition
0,Thriller_Bark_2_0,3
1,Straw_Hat_Pirates_18_0,0
2,Laffitte_10_0,0
3,Killer_9_0,6
4,Zeff_34_0,3
...,...,...
4530,Buggy%27s_Delivery_4_0,1
4531,Belladonna_6_0,1
4532,Charlotte_Pudding_33_0,0
4533,Maria_Napole_5_0,8


In [26]:
import graphistry

graphistry.register( 
        api=3,
        personal_key_id="35ODTI9CO5",
        personal_key_secret="2JHUSJENWLQO1KVE"
    )

g = graphistry.edges(edges_df, 'src', 'dst')
g = g.nodes(nodes_df, 'node')
g = g.bind(point_title='node') 
g = g.modularity_weighted_layout("partition")

g.plot()